# MNIST

-2개의 히든 레이어를 도입한다.

-input layer는 28*28개의 노드

-hidden layer의 노드의 수는 자유롭게 설정해도 좋다.

-output은 10개(0~9)로 설정한다.

test data를 예측하여 정확도를 최대가 되도록 학습하여라.

-제출 시 test data의 정확도를 출력하여 제출하세요.

In [178]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision

train_data = torchvision.datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = torchvision.transforms.ToTensor(), 
    download = True,            
)
test_data = torchvision.datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = torchvision.transforms.ToTensor()
)

train_data

test_data # 제출시 이 test 데이터에 대한 정확도를 출력하세요.

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [179]:
X=torch.flatten(train_data.data,start_dim=1).float()
y=train_data.targets
X=X/255 # 데이터를 0~1로 정규화

X_test=torch.flatten(test_data.data,start_dim=1).float()
y_test=test_data.targets
X_test=X_test/255 # 데이터를 0~1로 정규화

In [180]:
din, dh1, dh2, dout = 28*28,200,100,10
model = torch.nn.Sequential(
    torch.nn.Linear(din,dh1), # input - hidden1
    torch.nn.ReLU(),         # activation function of hidden layer 
    torch.nn.Linear(dh1,dh2), # input - hidden1
    torch.nn.ReLU(),         # activation function of hidden layer 
    torch.nn.Linear(dh2,dout), # hidden - out
    torch.nn.Softmax(dim=1)
)

# loss and optimizer
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [183]:
# learning
nums_epoch = 501
def train(model, criterion, optimizer, nums_epoch, X,y,X_test,y_test):
    for epoch in range(nums_epoch):
        y_pred = model(X)
        y_test_pred = model(X_test)
        
        loss = criterion(y_pred,y)
        optimizer.zero_grad() # zero_grad(): 미분값 초기화
        loss.backward() # backpropagation
        optimizer.step() # update weights and biases
        
        if(epoch%10==0):
            prediction = y_pred.max(1)[1] # max indices
            corrects = (prediction == y)
            accuracy = corrects.sum().float() / float( y.size(0) )
               
            prediction_test = y_test_pred.max(1)[1]
            corrects_test = (prediction_test == y_test)
            accuracy_test = corrects_test.sum().float() / float( y_test.size(0) )
            print(epoch,"loss=", loss.item(), "accu=", accuracy.item(), "accu_test=", accuracy_test.item())

In [187]:
train(model,criterion, optimizer, nums_epoch, X, y,X_test,y_test);

0 loss= 1.4723237752914429 accu= 0.9888666868209839 accu_test= 0.9710000157356262
10 loss= 1.4723056554794312 accu= 0.9888833165168762 accu_test= 0.9708999991416931
20 loss= 1.4722708463668823 accu= 0.9889166951179504 accu_test= 0.9708999991416931
30 loss= 1.4722200632095337 accu= 0.9890000224113464 accu_test= 0.9708999991416931
40 loss= 1.472187876701355 accu= 0.9890166521072388 accu_test= 0.9708999991416931
50 loss= 1.4721606969833374 accu= 0.9890499711036682 accu_test= 0.9708999991416931
60 loss= 1.472141146659851 accu= 0.9890499711036682 accu_test= 0.9707000255584717
70 loss= 1.472123146057129 accu= 0.9890833497047424 accu_test= 0.9708999991416931
80 loss= 1.4720960855484009 accu= 0.9890999794006348 accu_test= 0.9708999991416931
90 loss= 1.4720717668533325 accu= 0.9891166687011719 accu_test= 0.9707000255584717
100 loss= 1.472034215927124 accu= 0.9891499876976013 accu_test= 0.9710000157356262
110 loss= 1.4720045328140259 accu= 0.9891999959945679 accu_test= 0.9710000157356262
120 los

500번 돌리고 오버피팅이 나서 정확도가 떨어지는줄 알고 멈추려고 했는데 갑자기 다시 정확도가 오르기 시작했다.

그 뒤로 계속오른다.어디까지오르나 계속 돌려봤다.

1000번 근처부터는 정확도가 오르지않고 근처에 머무는 줄 알았는데 다시 평균적으로 정확도가 미미하게 오르기는 하지만 오르는 양이 점점 감소한다.

이 모델은 아무리 학습해도 97% 근처가 최고 정확도 인 것으로 보인다.

1500번 돌리니 1000번보다 정확도가 0.1%상승했다

2000번 때부터는 계속 97%근처값이 나온다 평균 증가량이 점점줄어든다. 더 이상의 학습은 무의미해보인다.
